In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split



In [2]:
data = pd.read_csv("./dataset.csv")



In [3]:
y = LabelEncoder().fit_transform(data["No-show"])



X = data[["Age", "Gender", "Scholarship",	"Hipertension",	"Diabetes",	"Alcoholism",	"Handcap",	"SMS_received"]]
X['Gender'] = LabelEncoder().fit_transform(X['Gender'])
X.head()

y = y.reshape((-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)
X_train = X_train.to_numpy()



C:\Users\SUDARSHAN\AppData\Local\Temp\ipykernel_30288\3618777564.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Gender'] = LabelEncoder().fit_transform(X['Gender'])


In [4]:
from main import Sequential, Dense
model = Sequential([
    Dense(8),
    Dense(4),
    Dense(1, activation='sigmoid')
])

In [5]:
model.compile(loss='binary_cross_entropy')

In [6]:
X_train.shape

(77368, 8)

In [14]:
model.fit(X_train, y_train, epochs=1000, learning_rate=1e-5)

Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320372869682575
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320372468444636
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320372067209166
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320371665976167
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320371264745631
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320370863517565
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320370462291965
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320370061068829
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.532036965984816
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320369258629957
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320368857414216
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320368456200941
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320368054990132
Accuracy: 0.7965954916761452, binary_cross_entropy: 0.5320367653781783
Accurac